Microservices - REST (Synchron)
-------------------------------

![](images/Microservices-REST.png)

Quelle: Buch Microservices Rezepte
- - -

Das Beispiel besteht aus drei Microservices: **Order**, **Customer** und **Catalog**. 

**Order** nutzt **Catalog** und **Customer** mit der REST-Schnittstelle. Ausserdem bietet jeder Microservice einige HTML-Seiten an.

Zusätzlich ist im Beispiel ein Apache-Webserver installiert, der dem Benutzer mit einer Webseite einen einfachen Einstieg in das System ermöglicht.

In [3]:
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/apache.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/catalog.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/customer.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/order.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/postgres.yaml
! kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service apache --namespace ms-kubernetes -o=jsonpath='{ .spec.ports[0].nodePort }')/"   


service/apache unchanged
deployment.apps/apache unchanged
service/catalog unchanged
deployment.apps/catalog unchanged
service/customer unchanged
deployment.apps/customer unchanged
service/order unchanged
deployment.apps/order unchanged
service/postgres unchanged
deployment.apps/postgres unchanged
http://192.168.3.82:32280

Das Microservices Zusammenspiel verfolgen wird in der Weave Scope Oberfläche.

Mittels den nachfolgenden Befehlen wird der Port der Weave Scope Oberfläche gegen aussen (IP VM) freigeschaltet.

In [4]:
! kubectl get service weave-scope-app -n weave -o yaml | sed 's/ClusterIP/NodePort/' | kubectl apply -f -
! echo $(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service weave-scope-app --namespace weave -o=jsonpath='{ .spec.ports[0].nodePort }')/")'/#!/state/{"topologyId":"containers","topologyOptions":{"containers":{"namespace":["ms-kubernetes"]}}}'

service/weave-scope-app configured
http://192.168.3.82:31799/#!/state/{"topologyId":"containers","topologyOptions":{"containers":{"namespace":["ms-kubernetes"]}}}


***
### REST Paradigma

Representational State Transfer (abgekürzt REST, seltener auch ReST) bezeichnet ein Programmierparadigma für verteilte Systeme.

REST ist eine Abstraktion der Struktur und des Verhaltens des World Wide Web (HTTP GET, PUT, POST, DELETE).

REST hat das Ziel, einen Architekturstil zu schaffen, der die Anforderungen des modernen Web besser darstellt. Dabei unterscheidet sich REST vor allem in der Forderung nach einer einheitlichen Kommunikation Schnittstelle von anderen Architekturstilen.

Um das REST Paradigma zu verdeutlichen, führen wir mittels `curl` (Windows `Invoke-WebRequest`) mehrere API Aufrufe gegen unsere Microservices aus:

In [ ]:
! curl http://apache.ms-kubernetes/customer/customer

In [ ]:
! curl http://apache.ms-kubernetes/catalog/catalog

In [ ]:
! curl http://apache.ms-kubernetes/order/order/<Ersetzen mit order id>

***
### ReplicaSet

Um die ReplicaSet Funktionalität zu Demonstrieren, setzen wir die Anzahl der Pods `customer, order, catalog` auf 3 und schauen uns das Ergebnis an.

In [11]:
! kubectl --namespace rs scale --replicas=3 deployment/customer --namespace ms-kubernetes
! kubectl --namespace rs scale --replicas=3 deployment/order --namespace ms-kubernetes
! kubectl --namespace rs scale --replicas=3 deployment/catalog --namespace ms-kubernetes
! kubectl get pods --namespace ms-kubernetes

deployment.apps/customer scaled
deployment.apps/order scaled
deployment.apps/catalog scaled
NAME                        READY   STATUS    RESTARTS   AGE
apache-7857897746-ttw2l     1/1     Running   0          9m44s
catalog-fb5d5cddd-fj5jw     1/1     Running   0          5m49s
catalog-fb5d5cddd-hczld     1/1     Running   2          9m43s
catalog-fb5d5cddd-qwdfl     1/1     Running   0          5m49s
customer-564f7b76b9-47p48   1/1     Running   0          5m51s
customer-564f7b76b9-bk2fk   1/1     Running   1          9m41s
customer-564f7b76b9-j92rf   1/1     Running   0          5m51s
order-84b7bfc75c-7kpfv      1/1     Running   1          9m40s
order-84b7bfc75c-7kv2r      1/1     Running   0          5m50s
order-84b7bfc75c-fff7m      1/1     Running   0          5m50s
postgres-76f86d9567-dp65z   1/1     Running   0          9m38s


***
### Deployments

Das Menu vom `apache` Web Server zeigt immer noch das Hystrix Dashboard an. Dieses wird in einem Kubernetes Umfeld nicht mehr benötigt. Deshalb ersetzen wir das Container Image durch die neuste Version.


In [18]:
! kubectl set image deployment/apache apache=ewolff/microservice-kubernetes-demo-apache:latest --namespace ms-kubernetes

deployment.apps/apache image updated


In [19]:
! kubectl describe deployment/apache --namespace ms-kubernetes

Name:                   apache
Namespace:              ms-kubernetes
CreationTimestamp:      Mon, 25 Jan 2021 09:28:14 +0000
Labels:                 <none>
Annotations:            deployment.kubernetes.io/revision: 3
Selector:               app=apache
Replicas:               1 desired | 1 updated | 1 total | 1 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  app=apache
           group=web
           tier=frontend
  Containers:
   apache:
    Image:        ewolff/microservice-kubernetes-demo-apache:latest
    Port:         80/TCP
    Host Port:    0/TCP
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Available      True    MinimumReplicasAvailable
  Progressing    True    NewReplicaSetAvailable
OldReplicaSets:  <none>
NewReplicaSet:   apache-cbb4bdff6 (1/1 rep

- - -

Aufräumen

In [ ]:
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/apache.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/catalog.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/customer.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/hystrix.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/order.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/postgres.yaml